# PostgreSQL 개발을 위한 필수 기본 지식

PostgreSQL은 강력한 오픈소스 객체-관계형 데이터베이스 시스템(ORDBMS)으로, 엔터프라이즈급 기능을 제공합니다.

## 1. PostgreSQL 기본 아키텍처

- **클라이언트/서버 모델**: 여러 클라이언트가 단일 서버 인스턴스에 연결
- **프로세스 기반 아키텍처**: 각 연결에 대해 별도의 서버 프로세스 생성
- **MVCC(Multi-Version Concurrency Control)**: 동시성 제어 메커니즘

## 2. 설치 및 설정

### 2.1 주요 설치 방법
```bash
# Ubuntu
sudo apt-get update
sudo apt-get install postgresql postgresql-contrib

# Mac (Homebrew)
brew install postgresql
brew services start postgresql

# Windows: EnterpriseDB에서 설치 패키지 다운로드
```

### 2.2 기본 명령어
```bash
# 서비스 시작/중지
sudo service postgresql start
sudo service postgresql stop

# psql 접속
sudo -u postgres psql

# 데이터베이스 생성
CREATE DATABASE mydb;

# 사용자 생성 및 권한 부여
CREATE USER myuser WITH PASSWORD 'mypassword';
GRANT ALL PRIVILEGES ON DATABASE mydb TO myuser;
```

## 3. 기본 SQL 문법

### 3.1 테이블 생성
```sql
CREATE TABLE users (
    id SERIAL PRIMARY KEY,
    username VARCHAR(50) UNIQUE NOT NULL,
    email VARCHAR(255) UNIQUE NOT NULL,
    created_at TIMESTAMP WITH TIME ZONE DEFAULT CURRENT_TIMESTAMP,
    is_active BOOLEAN DEFAULT TRUE
);
```

### 3.2 CRUD 연산
```sql
-- 삽입
INSERT INTO users (username, email) 
VALUES ('john_doe', 'john@example.com');

-- 조회
SELECT * FROM users WHERE is_active = TRUE;

-- 업데이트
UPDATE users SET is_active = FALSE WHERE id = 1;

-- 삭제
DELETE FROM users WHERE id = 1;
```

## 4. 고급 데이터 타입

### 4.1 JSON/JSONB
```sql
CREATE TABLE products (
    id SERIAL PRIMARY KEY,
    name VARCHAR(100),
    attributes JSONB
);

-- JSONB 연산자
SELECT * FROM products WHERE attributes->>'color' = 'red';
```

### 4.2 배열 타입
```sql
CREATE TABLE posts (
    id SERIAL PRIMARY KEY,
    title VARCHAR(100),
    tags VARCHAR(50)[]
);

-- 배열 검색
SELECT * FROM posts WHERE 'database' = ANY(tags);
```

### 4.3 기타 특수 타입
- **UUID**: 고유 식별자
- **GIS 타입**: PostGIS 확장으로 지리공간 데이터 처리
- **범위 타입**: 숫자, 날짜, 타임스탬프 범위 저장

## 5. 인덱스와 성능 최적화

### 5.1 인덱스 유형
```sql
-- B-tree 인덱스 (기본)
CREATE INDEX idx_users_username ON users(username);

-- 부분 인덱스
CREATE INDEX idx_active_users ON users(username) WHERE is_active = TRUE;

-- 다중 컬럼 인덱스
CREATE INDEX idx_users_name_email ON users(username, email);

-- GIN 인덱스 (JSONB, 배열)
CREATE INDEX idx_products_attributes ON products USING GIN(attributes);
```

### 5.2 EXPLAIN 분석
```sql
EXPLAIN ANALYZE SELECT * FROM users WHERE username = 'john_doe';
```

## 6. 트랜잭션과 동시성 제어

```sql
-- 트랜잭션 예제
BEGIN;
UPDATE accounts SET balance = balance - 100 WHERE id = 1;
UPDATE accounts SET balance = balance + 100 WHERE id = 2;
COMMIT;
-- 또는 ROLLBACK;

-- 격리 수준 설정
SET TRANSACTION ISOLATION LEVEL READ COMMITTED;
```

## 7. 저장 프로시저와 함수

### 7.1 PL/pgSQL 함수
```sql
CREATE OR REPLACE FUNCTION transfer_funds(
    sender_id INT,
    receiver_id INT,
    amount DECIMAL
) RETURNS BOOLEAN AS $$
BEGIN
    IF (SELECT balance FROM accounts WHERE id = sender_id) < amount THEN
        RETURN FALSE;
    END IF;
    
    UPDATE accounts SET balance = balance - amount WHERE id = sender_id;
    UPDATE accounts SET balance = balance + amount WHERE id = receiver_id;
    
    RETURN TRUE;
END;
$$ LANGUAGE plpgsql;
```

### 7.2 트리거
```sql
CREATE OR REPLACE FUNCTION update_timestamp()
RETURNS TRIGGER AS $$
BEGIN
    NEW.updated_at = NOW();
    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE TRIGGER update_user_timestamp
BEFORE UPDATE ON users
FOR EACH ROW EXECUTE FUNCTION update_timestamp();
```

## 8. 확장 기능

```sql
-- 확장 목록 보기
SELECT * FROM pg_available_extensions;

-- 확장 설치 (예: PostGIS)
CREATE EXTENSION IF NOT EXISTS postgis;

-- UUID 확장
CREATE EXTENSION IF NOT EXISTS "uuid-ossp";
SELECT uuid_generate_v4();
```

## 9. 백업 및 복구

```bash
# 덤프 생성
pg_dump -U username -d dbname -f backup.sql

# 덤프 복원
psql -U username -d dbname -f backup.sql

# 전체 클러스터 백업 (바이너리)
pg_basebackup -D /backup/location -U replicator -P -v
```

## 10. 연결 풀링과 성능 튜닝

### 10.1 주요 설정 파라미터 (`postgresql.conf`)
- `shared_buffers`: 일반적으로 시스템 메모리의 25%
- `work_mem`: 복잡한 정렬 작업에 사용
- `maintenance_work_mem`: 유지보수 작업에 사용
- `effective_cache_size`: 쿼리 플래너를 위한 설정

### 10.2 연결 풀링 도구
- **pgBouncer**: 경량 연결 풀러
- **Pgpool-II**: 더 복잡한 풀링 및 로드 밸런싱

## 11. 보안 모범 사례

1. **최소 권한 원칙**: 사용자에게 필요한 최소 권한만 부여
2. **SSL 연결**: `postgresql.conf`에서 `ssl = on` 설정
3. **암호화**: `pgcrypto` 확장 사용
4. **감사 로깅**: `log_statement = 'all'` (프로덕션에서는 주의)

## 12. 모니터링과 유지보수

```sql
-- 활성 쿼리 확인
SELECT * FROM pg_stat_activity;

-- 테이블 통계
SELECT * FROM pg_stat_user_tables;

-- 인덱스 사용 통계
SELECT * FROM pg_stat_user_indexes;

-- Vacuum 작업
VACUUM (VERBOSE, ANALYZE) users;
```

## 13. 애플리케이션 연결 예제 (Node.js)

```javascript
const { Pool } = require('pg');

const pool = new Pool({
  user: 'dbuser',
  host: 'database.server.com',
  database: 'mydb',
  password: 'secretpassword',
  port: 5432,
});

async function getUsers() {
  const client = await pool.connect();
  try {
    const res = await client.query('SELECT * FROM users WHERE active = $1', [true]);
    return res.rows;
  } finally {
    client.release();
  }
}
```

PostgreSQL은 다양한 고급 기능을 제공하는 동시에 SQL 표준을 잘 준수하는 데이터베이스 시스템입니다. 기본적인 SQL 지식 위에 PostgreSQL 특유의 기능들을 점진적으로 익히고, EXPLAIN ANALYZE를 활용한 쿼리 최적화와 적절한 인덱스 설계에 집중하는 것이 중요합니다.